## Face Dataset Collection

In [7]:
import cv2
import os

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Create dataset directory if not exists
DATASET_DIR = './dataset'
os.makedirs(DATASET_DIR, exist_ok=True)

# Initialize webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not access the webcam.")
    exit()

while True:
    person_name = input("Enter the person's name (or type 'done' to finish): ").strip()
    if person_name.lower() == 'done':
        print("Dataset collection complete!")
        break

    person_dir = os.path.join(DATASET_DIR, person_name)
    if os.path.exists(person_dir):
        print(f"Images for {person_name} already exist. Skipping...")
        continue
    os.makedirs(person_dir, exist_ok=True)

    print(f"Collecting images for {person_name}. Press 'q' to stop.")
    count = 0

    while count < 100:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame from webcam.")
            break

        # Convert to grayscale for better face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

        for (x, y, w, h) in faces:
            padding = 10
            x1, y1 = max(0, x - padding), max(0, y - padding)
            x2, y2 = min(frame.shape[1], x + w + padding), min(frame.shape[0], y + h + padding)

            # Draw rectangle around face
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Save the cropped face
            face_roi = frame[y1:y2, x1:x2]
            img_path = os.path.join(person_dir, f"{person_name}_{count}.jpg")
            cv2.imwrite(img_path, face_roi)
            count += 1
            print(f"Captured {count}/100 images for {person_name}")

            if count >= 100:
                break

        # Display the frame
        cv2.imshow("Face Capture - Press 'q' to Quit", frame)

        # Quit when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Enter the person's name (or type 'done' to finish):  Ali


Captured 1/100 images for Ali
Captured 2/100 images for Ali
Captured 3/100 images for Ali
Captured 4/100 images for Ali
Captured 5/100 images for Ali
Captured 6/100 images for Ali
Captured 7/100 images for Ali
Captured 8/100 images for Ali
Captured 9/100 images for Ali
Captured 10/100 images for Ali
Captured 11/100 images for Ali
Captured 12/100 images for Ali
Captured 13/100 images for Ali
Captured 14/100 images for Ali
Captured 15/100 images for Ali
Captured 16/100 images for Ali
Captured 17/100 images for Ali
Captured 18/100 images for Ali
Captured 19/100 images for Ali
Captured 20/100 images for Ali
Captured 21/100 images for Ali
Captured 22/100 images for Ali
Captured 23/100 images for Ali
Captured 24/100 images for Ali
Captured 25/100 images for Ali
Captured 26/100 images for Ali
Captured 27/100 images for Ali
Captured 28/100 images for Ali
Captured 29/100 images for Ali
Captured 30/100 images for Ali
Captured 31/100 images for Ali
Captured 32/100 images for Ali
Captured 33/100 i

Enter the person's name (or type 'done' to finish):  done


Dataset collection complete!


## Data Preprocessing

In [8]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

IMG_SIZE = 128  # Image size for input
DATA_DIR = './dataset'

# Get class labels from dataset folder names
persons = [name for name in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, name))]
labels = {name: i for i, name in enumerate(persons)}

# Load dataset images
X, y = [], []
for person in persons:
    folder = os.path.join(DATA_DIR, person)
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        try:
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                X.append(img)
                y.append(labels[person])
        except Exception as e:
            print(f"Error processing image {img_path}: {e}")

X = np.array(X) / 255.0  # Normalize images
y = np.array(y)

if len(X) == 0:
    print("No valid data found. Exiting...")
    exit()

# Split dataset into training & validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = to_categorical(y_train, num_classes=len(persons))
y_val = to_categorical(y_val, num_classes=len(persons))

print("Dataset preprocessed successfully!")


Dataset preprocessed successfully!


## Model Training with Transfer Learning

In [9]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

# Load the preprocessed data
IMG_SIZE = 128  # Image size used in preprocessing
NUM_CLASSES = len(persons)  # Number of classes (persons)

# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=16)

# Save the model
model.save('attendance_resnet_model.h5')
print("ResNet50 model trained and saved successfully!")


Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 264ms/step - accuracy: 0.5052 - loss: 0.7422 - val_accuracy: 0.4750 - val_loss: 0.6728
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 149ms/step - accuracy: 0.6431 - loss: 0.6528 - val_accuracy: 0.8750 - val_loss: 0.6321
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 150ms/step - accuracy: 0.6350 - loss: 0.6176 - val_accuracy: 0.8250 - val_loss: 0.6019
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 150ms/step - accuracy: 0.6684 - loss: 0.5669 - val_accuracy: 0.6000 - val_loss: 0.5930
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 151ms/step - accuracy: 0.6964 - loss: 0.5555 - val_accuracy: 0.8250 - val_loss: 0.5625
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 158ms/step - accuracy: 0.7627 - loss: 0.5542 - val_accuracy: 0.8750 - val_loss: 0.5324
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 158ms/step - accuracy: 0.7943 - loss: 0.5141 - val_accuracy: 0.8750 - val_loss: 0.5113
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 154ms/step - accuracy: 0.8687 - loss: 0.4699 - val_accuracy: 0.

ResNet50 model trained and saved successfully!


## Real-Time Face Recognition

In [10]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load trained model
model = load_model('attendance_resnet_model.h5')

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Define image size
IMG_SIZE = 128

# Start webcam
cap = cv2.VideoCapture(0)

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame from webcam.")
            break

        # Convert frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

        for (x, y, w, h) in faces:
            # Draw rectangle around the detected face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Preprocess face for model
            face_roi = frame[y:y+h, x:x+w]
            face_roi = cv2.resize(face_roi, (IMG_SIZE, IMG_SIZE))
            face_roi = np.expand_dims(face_roi, axis=0) / 255.0

            # Predict person
            predictions = model.predict(face_roi)
            person_index = np.argmax(predictions)
            confidence = np.max(predictions)

            label = list(labels.keys())[person_index] if confidence > 0.7 else "Unidentified"

            # Display result
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        cv2.imshow("Face Recognition - Press 'Q' to Quit", frame)

        # Check for quit command
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):  # Press 'q' to exit
            print("\n[INFO] Quitting application...")
            break

except KeyboardInterrupt:
    print("\n[INFO] Keyboard Interrupt detected. Exiting gracefully...")

finally:
    # Release resources and close windows
    cap.release()
    cv2.destroyAllWindows()
    print("[INFO] Resources released. Application closed.")


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'attendance_transfer_learning_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)